<h1>Amazon Dataset Processing</h1>

<h3>1. Function to return: Author, ISBN-10, ISBN-13, platform, price, category1, category2, description</h3>

Update From <b>Akshay</b> - 
<font color='red'>Information available from Kaggle:</font>

<li>ISBN 10</li>
<li>Author</li>
<li>Platform</li>

<b>Information Being scraped:</b>

<li>Description</li>
<li>Price</li>

Categories aren't available for all books. We might have to skip this for the moment.

<h3> Read data from amazon_com_extras. </h3>

In [76]:
import pandas as pd
import random 
import json
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import time

my_csvfile='./file2_lily.csv' #please modify filename to your own need
start_entry=1001
end_entry=1010 #akshay already did 1~1000 entry, edit yours accordingly 
#from time import sleep
#from random import random
#path_kaggle = r'C:\Users\endwy\Documents\Columbia MSBA\Fall 2018\E4501 - Tools for Analytics\Project\kaggle data\amazon_com_extras.csv'
path_kaggle = './amazon_com_extras.csv' #download from https://www.kaggle.com/ucffool/amazon-sales-rank-data-for-print-and-kindle-books/version/3

with open(path_kaggle):
    #Added fields since line 808 in csv has more than 6 cols and needs to be told to ignore extra data.
    
    fields=['ASIN','GROUP','FORMAT','TITLE','AUTHOR','PUBLISHER']
    df_kaggle = pd.read_csv(path_kaggle, encoding='latin1', dtype=str,usecols=fields)

In [77]:
#print(len(df_kaggle))
Df_read = df_kaggle.iloc[start_entry:end_entry+1]
# df_kaggle
# Df_read['ddd']=''
# Df_read['ddd'].iloc[1]=2
Df_read

,ASIN,GROUP,FORMAT,TITLE,AUTHOR,PUBLISHER
1001,1772265039,book,hardcover,The Preschooler's Handbook: Bilingual (English...,"Dayna Martin, A.R. Roumanis",Engage Books
1002,0618945393,book,hardcover,Mariposa Road: The First Butterfly Big Year,Robert Michael Pyle,Houghton Mifflin Harcourt
1003,1493210033,book,hardcover,Implementing SAP BW on SAP HANA,"Matthias Merz, Torben HÃ¼gens, Steve Blum",SAP Press
1004,022629899X,book,hardcover,Plundered Skulls and Stolen Spirits: Inside th...,Chip Colwell,University Of Chicago Press
1005,1501176684,book,hardcover,Love Her Wild: Poems,Atticus,Atria Books
1006,0307987302,book,hardcover,Rod: The Autobiography,Rod Stewart,Crown Archetype
1007,1510737480,book,hardcover,The Benefits of Being an Octopus,Ann Braden,Sky Pony Press
1008,1481411225,book,hardcover,Compulsion (Heirs of Watson Island),Martina Boone,Simon Pulse
1009,1941631606,book,hardcover,Media Circus: A Look at Private Tragedy in the...,"Kim Goldman, Tatsha Robertson",BenBella Books
1010,1501805339,book,hardcover,John: The Gospel of Light and Life (John series),Adam Hamilton,Abingdon Press


<h4>Note: Amazon will identify and block you as a bot, please exercise caution while pulling data. Use user agents and throttling to prevent sending multilpe requests</h4>

User Agent Code - Refer https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

Updated the following :

<li>Throttling</li>
<li>Saving data to a single dataframe</li>

<h3> 1. Pull data from the Kaggle Dataset. We'll be using ISBN 10 to scrape book descriptions</h3>

In [72]:
import requests
import numpy as np

def amazon_description(book_metadata):
    
    import random
    import time
    
    #New columns that will be added to book_metadata
    book_descriptions=list()
    book_price=list()
    book_rating=list()
    
    user_agent_list = [
           #Chrome
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            #Firefox
            'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
            'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
        ]
    
    
    book_metadata['Description']=''
    book_metadata['Price']=''
    book_metadata['Rating']=''
    for i in range(len(book_metadata)):
        
        #User Agents to prevent bot detection
        user_agent=random.choice(user_agent_list)
        book_isbn=book_metadata['ASIN'].iloc[i]
        
        print(book_isbn)
        base_url = "https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords="
        url=base_url+book_isbn
        print(url)
        headers={'User-Agent':user_agent}
        book_response=requests.get(url,headers=headers)
        if book_response.status_code != 200:
            print("Getting Book Details Failed")
            return None
        else:
            result_page=bs(book_response.content,'lxml')
            all_book_a_tags=result_page.find("a",class_="a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal")
            try:
                book_url=all_book_a_tags.get("href")
                description,price,rating=get_book_description(book_url,user_agent_list)
            except: 
                continue
            print(description,price,rating)
#             book_descriptions.append(description)
#             book_price.append(price)
#             book_rating.append(rating)
        book_metadata['Description'].iloc[i]=description
        book_metadata['Price'].iloc[i]=price
        book_metadata['Rating'].iloc[i]=rating
        book_metadata.to_csv(my_csvfile,encoding='utf-8') #save data after each query
#     book_metadata['Description']=book_descriptions
#     book_metadata['Price']=book_price
#     book_metadata['Rating']=book_rating
    
    return book_metadata
    



<h3> Write code to pull the following - </h3>
 
<h4> 
<li> Description </li>
<li> Rating </li>
<li> Price </li>
    
</h4>    

In [61]:
def get_book_description(url,user_agent_list):
    book_description=''
    book_rating=''
    book_price=''
    print(url)
    #User Agents to prevent bot detection
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    
    #Throttling to spoof the server into believe these are generated by users
    time.sleep(7)
    
    book_description_response=requests.get(url,headers=headers)
    if book_description_response.status_code!=200:
        print("Error in scraping description")
        return None
    else:
        book_page=bs(book_description_response.content,"lxml")
        a1=book_page.find('div',{'id':'a-page'})
        a2=a1.find('div',class_='book en_US').find('div',class_='a-container').find('div',{'id':'centerCol'})
        try:
            book_description=a2.find('div',{'id':'bookDescription_feature_div'}).find('noscript').find('div').get_text()
        except:
            book_description=np.NAN
        try:
            book_rating=a2.find('div',{'id':'averageCustomerReviews_feature_div'}).find('span',class_='a-declarative').find('span',{'id':'acrPopover'}).find('span',class_='a-declarative').find('a').find('i').find('span').get_text().split()[0]    
        except:
            book_rating=np.NAN
        try:
            book_price=a2.find('div',{'id':'MediaMatrix'}).find('div').find('div',{'id':'tmmSwatches'}).find('ul').find('li',class_='swatchElement selected').find('span').find('span').find('span').find('span',class_='a-color-base').find('span').get_text().strip()
        except:
            book_price=np.NAN
        return book_description,book_price,book_rating

<h3> You might need to batch the kaggle dataset into sets of 1000 rows. Takes a few hours to run</h3>

In [73]:
#Be very careful when running the function
df_final=amazon_description(Df_read) 
#print(random.__file__)

/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

1250150183
https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords=1250150183
https://www.amazon.com/Swamp-Washingtons-Murky-Corruption-Cronyism/dp/1250150183/ref=sr_1_1/144-7503819-0199013?s=books&ie=UTF8&qid=1543087649&sr=1-1&keywords=1250150183
 The Instant New York Times and USA Today Bestseller!When Washington D.C. was first built, it was on top of a swamp that had to be drained. Donald Trump says it's time to drain it again. In The Swamp, bestselling author Eric Bolling presents an infuriating, amusing, revealing, and outrageous history of American politics, past and present, Republican and Democrat. From national political scandals to tempests in a teapot that blew up; bribery, blackmail, bullying, and backroom deals that contradicted public policies; cronyism that cost taxpayers hundreds upon hundreds of millions of dollars; and personal conduct that can only be described as regrettable, The Swamp is a journey downriver through the bayous 

/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/LovelyCheuk/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


https://www.amazon.com/Rise-Shine-Benedict-Stone-Novel/dp/0778319997/ref=sr_1_1/130-0756217-0674452?s=books&ie=UTF8&qid=1543087658&sr=1-1&keywords=0778319997
 Phaedra Patrick's debut novel, The Curious Charms of Arthur Pepper, was hailed as "poignant" and "utterly endearing." Now she returns with Rise and Shine, Benedict Stone, a gem of a novel about family, forgiveness and one man's second chance at happiness. Moonstone for empathy. Azurite for memories. Lapis lazuli for truth In the quiet village of Noon Sun, Benedict Stone has settled into a complacent and predictable routine. Business at his jewelry shop has dried up; his marriage is on the rocks. His life is in desperate need of a jump start And then a surprise arrives at his door. Gemma is Benedict's audacious teenage niecethe daughter of his estranged brother, Charlie. The two Stone brothers had a falling out and haven't spoken in almost two decades, since Charlie left for America. Reckless and stubborn, Gemma invites herself

<h3> Save the file. </h3>

In [70]:
#df_final
df_final.to_csv('file2_lily.csv',encoding='utf-8')

,ASIN,GROUP,FORMAT,TITLE,AUTHOR,PUBLISHER,Description,Price,Rating
0,1250150183,book,hardcover,The Swamp: Washington's Murky Pool of Corrupti...,Eric Bolling,St. Martin's Press,The Instant New York Times and USA Today Best...,$5.95,4.5
1,0778319997,book,hardcover,"Rise and Shine, Benedict Stone: A Novel",Phaedra Patrick,Park Row Books,"Phaedra Patrick's debut novel, The Curious Ch...",$16.99,4.2
2,1608322564,book,hardcover,Sell or Be Sold: How to Get Your Way in Busine...,Grant Cardone,Greenleaf Book Group Press,Whether it's selling your company's product i...,$23.08,4.6


<h3>4. Rename ISBN-13 to ASIN for future merge</h3>

In [ ]:
# df_az.rename(columns={'ISBN-13':'ASIN'})

<h1>Kaggle Dataset Processing</h1>

<h4><font color='red'>Note:</font color='red'> you will need to change the path to your directory when you want to run/edit</h4>

<h3>1. File import</h3>

In [ ]:
import pandas as pd
#path_kaggle = r'C:\Users\endwy\Documents\Columbia MSBA\Fall 2018\E4501 - Tools for Analytics\Project\kaggle data\amazon_com_extras.csv'
path_kaggle = './amazon_com_extras.csv' #download from https://www.kaggle.com/ucffool/amazon-sales-rank-data-for-print-and-kindle-books/version/3

with open(path_kaggle):
    #Added fields since line 808 in csv has more than 6 cols and needs to be told to ignore extra data.
    
    fields=['ASIN','GROUP','FORMAT','TITLE','AUTHOR','PUBLISHER']
    df_kaggle = pd.read_csv(path_kaggle, encoding='latin1', dtype=str,usecols=fields)

<h3>3. Merge Amazon scrape andKkaggle dfs</h3>

In [ ]:
# merge = pd.merge(df_az,df_kaggle, on='ASIN',how='left')

<h3>4. Save out</h3>

In [ ]:
# merge.to_csv('C:\Users\endwy\Documents\Columbia MSBA\Fall 2018\E4501 - Tools for Analytics\Project\kaggle data\processed.csv', index=False)

<h1><font color='red'>Note: </font color='red'></h1><h2>Running the following will convert .ipynb to .py </h2>

In [ ]:
!jupyter nbconvert --to script AZscraping.ipynb